## Cargar y parsear archivo

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/AnaLuisaIA/antifungal-peptides/refs/heads/main/databases/DBAASP%20%5Bmultipep%5D/peptides_fungus.csv'

In [3]:
df = pd.read_csv(url)

In [5]:
df.columns

Index(['ID', 'COMPLEXITY', 'NAME', 'N TERMINUS', 'SEQUENCE', 'C TERMINUS',
       'SYNTHESIS TYPE', 'TARGET GROUP', 'TARGET OBJECT'],
      dtype='object')

In [4]:
display(df.head())

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT
0,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,NaN,NaN
1,14,Monomer,Brevinin-2 related peptide B2RP,NaN,GIWDTIKSMGKVFAGKILQNL,AMD,Ribosomal,NaN,NaN
2,15,Monomer,Histatin 5,NaN,DSHAKRHHGYKRKFHEKHHSHRGY,NaN,Ribosomal,NaN,NaN
3,37,Monomer,PGLa,NaN,GMASKAGAIAGKIAKVALKAL,AMD,Ribosomal,NaN,NaN
4,38,Monomer,NaN,C16,KAaaKWAaKaAK,AMD,Synthetic,NaN,NaN


In [18]:
df.shape

(4988, 9)

In [19]:
df_afp = pd.DataFrame({
    "Sequence": df["SEQUENCE"],
    "label": 1
})

In [20]:
display(df_afp.head())

,Sequence,label
0,KVvvKWVvKvVK,1
1,GIWDTIKSMGKVFAGKILQNL,1
2,DSHAKRHHGYKRKFHEKHHSHRGY,1
3,GMASKAGAIAGKIAKVALKAL,1
4,KAaaKWAaKaAK,1


In [21]:
df_afp.shape

(4988, 2)

## Verificar duplicados

In [22]:
df_afp["Sequence"].unique().shape

(4484,)

In [23]:
df_afp = df_afp.drop_duplicates().copy()

In [24]:
df_afp.shape

(4484, 2)

## Verificar canónicos

In [25]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [26]:
# Verificar cuáles secuencias válidas
df_afp["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df_afp["Sequence"]]

In [27]:
no_validas = df_afp[~df_afp["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 658


In [28]:
df_validas = df_afp[df_afp["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 3826


In [30]:
display(df_validas)

,Sequence,label
1,GIWDTIKSMGKVFAGKILQNL,1
2,DSHAKRHHGYKRKFHEKHHSHRGY,1
3,GMASKAGAIAGKIAKVALKAL,1
6,KWLNALLHHGLNCAKGVLA,1
7,VTCDILSVEAKGVKLNDAACAAHCLFRGRSGGYCNGKRVCVCR,1
...,...,...
4983,KLLKLVKKVVGALG,1
4984,KLLKLLKKVVGALG,1
4985,KLLKLLKKLVGALG,1
4986,KLLKLLKKLLGALG,1


In [38]:
df_validas.to_csv('DBAASP_[multipep]_labeled.csv', index=False, header=True)

## Obtener metadatos

In [31]:
import json

In [32]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [33]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [34]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [35]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "DBAASP [multipep]")

In [36]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [37]:
export_json("metadata.json", dict_metadata)